खालील नोटबुक चालवण्यासाठी, जर तुम्ही अद्याप केले नसेल, तर तुम्हाला `text-embedding-ada-002` या बेस मॉडेलचा वापर करणारा एक मॉडेल डिप्लॉय करावा लागेल आणि .env फाईलमध्ये डिप्लॉयमेंटचे नाव `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` म्हणून सेट करावे लागेल.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

पुढे, आपण एम्बेडिंग इंडेक्स पांडस डेटाफ्रेममध्ये लोड करणार आहोत. एम्बेडिंग इंडेक्स `embedding_index_3m.json` नावाच्या JSON फाईलमध्ये संग्रहित आहे. एम्बेडिंग इंडेक्समध्ये ऑक्टोबर 2023 च्या शेवटपर्यंत प्रत्येक यूट्यूब ट्रान्सक्रिप्टसाठी एम्बेडिंग्ज असतात.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

पुढे, आपण `get_videos` नावाची एक फंक्शन तयार करणार आहोत जी क्वेरीसाठी Embedding Index मध्ये शोध करेल. ही फंक्शन क्वेरीशी सर्वात जास्त सादृश्य असलेली टॉप 5 व्हिडिओ परत करेल. फंक्शन खालीलप्रमाणे कार्य करते:

1. प्रथम, Embedding Index ची एक कॉपी तयार केली जाते.
2. नंतर, OpenAI Embedding API वापरून क्वेरीसाठी Embedding गणना केली जाते.
3. त्यानंतर Embedding Index मध्ये `similarity` नावाचा एक नवीन कॉलम तयार केला जातो. `similarity` कॉलममध्ये क्वेरी Embedding आणि प्रत्येक व्हिडिओ सेगमेंटसाठी Embedding यांच्यातील कोसाइन सादृश्यता असते.
4. पुढे, `similarity` कॉलमद्वारे Embedding Index फिल्टर केला जातो. Embedding Index मध्ये फक्त असे व्हिडिओ समाविष्ट केले जातात ज्यांची कोसाइन सादृश्यता 0.75 किंवा त्याहून जास्त असते.
5. शेवटी, Embedding Index `similarity` कॉलमनुसार क्रमवारी लावला जातो आणि टॉप 5 व्हिडिओ परत केले जातात.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ही फंक्शन खूप सोपी आहे, ती फक्त शोध क्वेरीचे निकाल छापते.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. प्रथम, एम्बेडिंग निर्देशांक पांडस डेटाफ्रेममध्ये लोड केला जातो.
2. नंतर, वापरकर्त्याला क्वेरी टाकण्यास सांगितले जाते.
3. त्यानंतर `get_videos` फंक्शन कॉल करून क्वेरीसाठी एम्बेडिंग निर्देशांक शोधला जातो.
4. शेवटी, `display_results` फंक्शन कॉल करून वापरकर्त्यास निकाल दाखवले जातात.
5. नंतर वापरकर्त्याला आणखी एक क्वेरी टाकण्यास सांगितले जाते. हा प्रक्रिया वापरकर्ता `exit` टाकेपर्यंत चालू राहते.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc.mr.png)

आपल्याला क्वेरी टाकण्यास सांगितले जाईल. एक क्वेरी टाका आणि एंटर दाबा. अ‍ॅप्लिकेशन क्वेरीशी संबंधित व्हिडिओंची यादी परत करेल. अ‍ॅप्लिकेशन प्रश्नाच्या उत्तराचा व्हिडिओतील स्थानाचा दुवा देखील परत करेल.

येथे काही क्वेरी आहेत ज्यांचा प्रयत्न करू शकता:

- Azure Machine Learning म्हणजे काय?
- कॉन्व्होल्यूशनल न्यूरल नेटवर्क कसे कार्य करतात?
- न्यूरल नेटवर्क म्हणजे काय?
- मी Azure Machine Learning सह Jupyter Notebooks वापरू शकतो का?
- ONNX म्हणजे काय?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:
हा दस्तऐवज AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून अनुवादित केला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित अनुवादांमध्ये चुका किंवा अचूकतेची कमतरता असू शकते. मूळ दस्तऐवज त्याच्या स्थानिक भाषेत अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी अनुवाद शिफारसीय आहे. या अनुवादाच्या वापरामुळे उद्भवलेल्या कोणत्याही गैरसमजुती किंवा चुकीच्या अर्थलागी आम्ही जबाबदार नाही.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
